In [51]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import physical_constants
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
import pandas as pd

# ⚠️⚠️⚠️⚠️Script not working!

# Numerically solving for eigenstates and eigenvalues of an arbitrary 1D potential

Description: Obtain the energy eigenvalues $E_n$ and wavefunctions $\psi_n(r)$ for the anharmonic Morse potential (below). Note that value of the parameters correspond the hydrogen fluoride. Tabulate $E_n$ for $n=0$ to $5$, and plot the corresponding $\psi_n(r)$.

$$V = D_e [1-e^{-\alpha x}]^2$$
- **Equilibrium bond energy**:
    $$D=6.091\times10^{-19} \text{ J}$$
- **Equilibrium bond length**:
    $$r_0=9.109\times10^{-11} \text{ m}, \quad x=r-r_0$$
- **Force constant**:
    $$k=1.039\times10^{3} \ \text{J}\text{m}^{-2}, \quad \alpha=\sqrt{ k / 2D_e}$$

## Define constants in atomic units

In [52]:
# Physical constants in SI units
hbar = physical_constants['Planck constant over 2 pi'][0]  # J·s
m_e = physical_constants['electron mass'][0]  # kg
a_0 = physical_constants['Bohr radius'][0]  # meters
E_h = physical_constants['Hartree energy'][0]  # Joules

# Convert given parameters to atomic units
# Atomic units: hbar = 1, m_e = 1, a_0 = 1, E_h = 1

## Calculate $\alpha$ using the given $k$ and $D_e$
✨ Make sure to convert parameters to atomic units

In [53]:
# Given parameters in SI units
k_SI = 1.039e3         # J/m^2
D_e_SI = 6.091e-19     # J

# Convert D_e to Hartree (atomic unit of energy)
D_e = D_e_SI / E_h     # now in units of Hartree

# Convert the force constant k to atomic units
# k in atomic units: E_h / a_0^2
k = k_SI * (a_0**2) / E_h # now dimensionless

# Compute alpha in atomic units
alpha = np.sqrt(k / (2 * D_e))  
print(f"Alpha in atomic units = {alpha:.3f}")

Alpha in atomic units = 1.545


## Reduced mass, $\mu$ of H-F molecule

$$\mu = \frac{m_\text{H} m_\text{F}}{m_\text{H} + m_\text{F}}

In [54]:
# Atomic mass in atomic mass units
m_H_u = 1.00784     # atomic mass units
m_F_u = 18.998403   # atomic mass units

u = physical_constants['atomic mass constant'][0]   # kg

# Masses in kg
m_H = m_H_u * u
m_F = m_F_u * u
mu_SI = (m_H * m_F) / (m_H + m_F)

# Reduced mass in atomic mass units (mu / m_e)
mu = mu_SI / m_e
print(f"Reduced mass in atomic units: {mu:.3e}")

Reduced mass in atomic units: 1.745e+03


## Set up spatial grid

In [55]:
# Spatial range in atomic units (typically a few Bohr radii around equilibrium)
x_min = -5  # a_0
x_max = 5   # a_0
N = 500  # Increase N for better resolution
x = np.linspace(x_min, x_max, N)
dx = x[1] - x[0]

# Displacement from equilibrium position (already in atomic units)

## Calculate the potential $V(x)$ at each point on the grid

In [56]:
V = D_e * (1 - np.exp(-alpha * x))**2   # V in Hartree units

# Set minimum value
V_floor = 1e-6  # Choose an appropriate floor value
V = np.maximum(D_e * (1 - np.exp(-alpha * x))**2, V_floor)

## Set up the kinetic energy operator ⚠️

In [57]:
hbar = 1.0545718e-34    # Js

# Construct the second derivative operator (kinetic energy term)
T_coeff = (hbar**2) / (2 * mu * dx**2)
diagonal = -2 * np.ones(N) * T_coeff
off_diagonal = np.ones(N - 1) * T_coeff

# Assemble the kinetic energy matrix
from scipy.sparse import diags

T = diags([off_diagonal, diagonal, off_diagonal], offsets=[-1,0,1])

## Construct the Hamiltonian Matrix
Combine the kinetic and potential energy terms

In [58]:
# Potential energy matrix (diagonal matrix)
V_matrix = diags(V, 0, format='csr')

# Hamiltonian matrix
H = T + V_matrix

# Convert to a dense matrix:
H = H.toarray()

Ensure no NaNs or infinities:

In [59]:
import numpy as np

if np.isnan(H.data).any() or np.isinf(H.data).any():
    print("Hamiltonian contains NaNs or Infinities.")
else:
    print("Hamiltonian matrix is valid.")

Hamiltonian matrix is valid.


Check the magnitudes of matrix elements:

In [60]:
max_element = np.max(np.abs(H.data))
min_element = np.min(np.abs(H.data[np.nonzero(H.data)]))
print(f"Max Hamiltonian element: {max_element}")
print(f"Min non-zero Hamiltonian element: {min_element}")

TypeError: only integer scalar arrays can be converted to a scalar index

## Solving the Schrodinger Equation ⚠️

In [61]:
from scipy.sparse.linalg import eigsh

# Number of eigenvalues and eigenvectors to compute
num_eigenvalues = 6

# Compute the lowest eigenvalues and corresponding eigenvectors
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA', tol=1e-5, maxiter=10000)

# 'which' parameter:
# 'SA' - compute the smallest algebraic eigenvalues
# 'SM' - may not work properly with sparse matrices and complex potentials

ArpackNoConvergence: ARPACK error -1: No convergence (10001 iterations, 0/6 eigenvectors converged)

## Converting Eigenvalues to Physical Units

In [ ]:
# Eigenvalues are already in Joules
E_n = eigenvalues   # J

## Normalizing the Eigenfunctions ⚠️
Normalize the eigenfunctions so that the integral over all space is 1

In [15]:
# Normalize eigenfunctions
psi_n = []
for i in range(num_eigenvalues):
    psi = eigenvectors[:, i]
    
    # Normalize
    norm = np.sqrt(np.sum(np.abs(psi)**2) + dx)    
    psi_n.append(psi / norm) 

## Tabulate the Energy Eigenvalues

In [16]:
import pandas as pd 

data = {'n': np.arange(num_eigenvalues), 'E_n (J)': E_n}
df = pd.DataFrame(data)
print(df)

   n       E_n (J)
0  0 -2.719156e-21
1  1 -1.348993e-21
2  2 -4.595187e-22
3  3  1.179863e-21
4  4  1.303045e-21
5  5  3.996451e-21


## Plot the Wavefunctions

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(10, 6))
for i in range(num_eigenvalues):
    plt.plot(x + r_0, psi_n[i] + E_n[i], label=f'n={i}')
    
# Plot the potential for reference
plt.plot(x + r_0, V, 'k-', label='Potential V(x)')

plt.xlabel('Position r (m)')
plt.ylabel('Energy (J)')
plt.title('Morse Potential Wavefunction')
plt.legend()
plt.show()

NameError: name 'r_0' is not defined

<Figure size 1000x600 with 0 Axes>